# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
from pprint import pprint
warnings.filterwarnings("ignore")
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
data = pd.read_csv("../output_data/cities.csv")
data.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [26]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = data[["Lat", "Lng"]]
humidity = data["Humidity"]
fg = gmaps.figure()
fg

#add the heatmap layer
heat_map = gmaps.heatmap_layer(locations,weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fg.add_layer(heat_map)

fg

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal

idealWeather = data.loc[(data["Max Temp"] >70) & (data["Max Temp"] < 80) & \
                        (data["Wind Speed"] < 10) & (data["Cloudiness"] == 0)].dropna()

idealWeather.head()
                        

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

# store into variable name hotel_df and set parameters to search for hotels
hotel_df = idealWeather[["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
444,birjand,IR,32.86,59.22,


In [31]:
# set parameters to search for hotel
params = {
    "radius": 5000, 
    "types": "hotel", 
    "key": g_key
}

# loop through dataframe
for index, row in hotel_df.iterrows():
    # lat and lng from current row
    lat = row['Lat']
    lng = row['Lng']
    # lat and lng to set locatin value in params dictionary
    params["location"] = f"{lat},{lng}"
    
    #base url to search using google
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #make request and print url
    name_address = requests.get(base_url, params=params).json()
    # pprint(name_address)

    try:
        #populate the hotel name
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except:
        print(f"An error has occurred for {row['City']}. Next...")
        pass
hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,Capitólio
88,cidreira,BR,-30.17,-50.22,CIDREIRA
138,morondava,MG,-20.30,44.28,Morondava
139,vaini,IN,15.34,74.49,Jagalbet
176,nador,MA,35.17,-2.93,Nador
297,mogok,MM,22.92,96.51,Mogok
444,birjand,IR,32.86,59.22,Birjand


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fg.add_layer(marker_layer)


# Display figure
fg

Figure(layout=FigureLayout(height='420px'))